In [1]:
import os
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
import numpy as np

# ✅ Ensure TF-DirectML works correctly
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"  # Disable OneDNN for DirectML

# ✅ Load Pretrained DenseNet
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# ✅ Freeze first 100 layers
for layer in base_model.layers[:100]:
    layer.trainable = False
for layer in base_model.layers[100:]:
    layer.trainable = True  # Unfreeze deeper layers

# ✅ Add Custom Layers
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.6)(x)  # Increased dropout to 0.6
output = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

# ✅ Use Adam Optimizer (Fix for DirectML)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

# ✅ Data Augmentation (Stronger Augmentations)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,  # Increased rotation
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,  # Added shear transformation
    zoom_range=0.2,  # Added zoom
    horizontal_flip=True,
    fill_mode='nearest'
)

# ✅ Load Data
train_generator = datagen.flow_from_directory(
    r'E:\multi_chronic_disease_project\dataset\train', 
    target_size=(224, 224), 
    batch_size=16, 
    class_mode='sparse'
)

val_generator = datagen.flow_from_directory(
    r'E:\multi_chronic_disease_project\dataset\validation', 
    target_size=(224, 224), 
    batch_size=16, 
    class_mode='sparse'
)

# ✅ Compute Class Weights for Imbalanced Data
class_counts = np.bincount(train_generator.classes)
total_samples = sum(class_counts)
class_weights = {i: total_samples / (5 * class_counts[i]) for i in range(5)}

# ✅ Learning Rate Scheduler (Manual Reduction)
def scheduler(epoch, lr):
    if epoch % 3 == 0:
        return lr * 0.5  # Reduce by half every 3 epochs
    return lr

lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

# ✅ Early Stopping & Checkpointing
callbacks = [
    lr_callback,
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(r"E:\multi_chronic_disease_project\model\densenet_best.h5", save_best_only=True)
]

# ✅ Compile Model
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# ✅ Train Model with Class Weights
model.fit(
    train_generator, 
    validation_data=val_generator, 
    epochs=15, 
    class_weight=class_weights, 
    callbacks=callbacks
)

# ✅ Save Final Model
model.save(r"E:\multi_chronic_disease_project\model\densenet_finetuned.h5")

# ✅ Check if DirectML GPU is available
print("Available GPUs:", tf.config.list_physical_devices('GPU'))


Found 22000 images belonging to 5 classes.
Found 2750 images belonging to 5 classes.
Epoch 1/15
1375/1375 [==============================] - 765s 546ms/step - loss: 0.6078 - accuracy: 0.7798 - val_loss: 0.3649 - val_accuracy: 0.8505 - lr: 5.0000e-05
Epoch 2/15
1375/1375 [==============================] - 708s 515ms/step - loss: 0.3448 - accuracy: 0.8674 - val_loss: 0.3151 - val_accuracy: 0.8738 - lr: 5.0000e-05
Epoch 3/15
1375/1375 [==============================] - 668s 486ms/step - loss: 0.2856 - accuracy: 0.8845 - val_loss: 0.4455 - val_accuracy: 0.8545 - lr: 5.0000e-05
Epoch 4/15
1375/1375 [==============================] - 886s 645ms/step - loss: 0.2309 - accuracy: 0.9040 - val_loss: 0.2534 - val_accuracy: 0.9029 - lr: 2.5000e-05
Epoch 5/15
1375/1375 [==============================] - 843s 613ms/step - loss: 0.2169 - accuracy: 0.9100 - val_loss: 0.3091 - val_accuracy: 0.8822 - lr: 2.5000e-05
Epoch 6/15
1375/1375 [==============================] - 696s 506ms/step - loss: 0.2024 - a

In [2]:
import os
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
import numpy as np

# ✅ Ensure TF-DirectML works correctly
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"  # Disable OneDNN for DirectML

# ✅ Set Model Save Path
checkpoint_path = r"E:\multi_chronic_disease_project\model\densenet_best.h5"

# ✅ Check if a saved model exists
if os.path.exists(checkpoint_path):
    print("Loading saved model...")
    model = tf.keras.models.load_model(checkpoint_path)
else:
    print("Training from scratch...")
    
    # ✅ Load Pretrained DenseNet
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # ✅ Freeze first 100 layers
    for layer in base_model.layers[:100]:
        layer.trainable = False
    for layer in base_model.layers[100:]:
        layer.trainable = True  # Unfreeze deeper layers

    # ✅ Add Custom Layers
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.6)(x)  # Increased dropout to 0.6
    output = Dense(5, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)

# ✅ Use Adam Optimizer (Fix for DirectML)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

# ✅ Data Augmentation (Stronger Augmentations)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,  # Increased rotation
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,  # Added shear transformation
    zoom_range=0.2,  # Added zoom
    horizontal_flip=True,
    fill_mode='nearest'
)

# ✅ Load Data
train_generator = datagen.flow_from_directory(
    r'E:\multi_chronic_disease_project\dataset\train', 
    target_size=(224, 224), 
    batch_size=16, 
    class_mode='sparse'
)

val_generator = datagen.flow_from_directory(
    r'E:\multi_chronic_disease_project\dataset\validation', 
    target_size=(224, 224), 
    batch_size=16, 
    class_mode='sparse'
)

# ✅ Compute Class Weights for Imbalanced Data
class_counts = np.bincount(train_generator.classes)
total_samples = sum(class_counts)
class_weights = {i: total_samples / (5 * class_counts[i]) for i in range(5)}

# ✅ Learning Rate Scheduler (Manual Reduction)
def scheduler(epoch, lr):
    if epoch % 3 == 0:
        return lr * 0.5  # Reduce by half every 3 epochs
    return lr

lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

# ✅ Early Stopping & Checkpointing
callbacks = [
    lr_callback,
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True)
]

# ✅ Compile Model (If not loaded from checkpoint)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# ✅ Train Model with Class Weights
model.fit(
    train_generator, 
    validation_data=val_generator, 
    epochs=15, 
    class_weight=class_weights, 
    callbacks=callbacks
)

# ✅ Save Final Model
final_model_path = r"E:\multi_chronic_disease_project\model\densenet_finetuned.h5"
model.save(final_model_path)

# ✅ Check if DirectML GPU is available
print("Available GPUs:", tf.config.list_physical_devices('GPU'))


Loading saved model...
Found 22000 images belonging to 5 classes.
Found 2750 images belonging to 5 classes.
Epoch 1/15
1375/1375 [==============================] - 725s 517ms/step - loss: 0.2357 - accuracy: 0.9020 - val_loss: 0.3523 - val_accuracy: 0.8742 - lr: 5.0000e-05
Epoch 2/15
1375/1375 [==============================] - 708s 515ms/step - loss: 0.2187 - accuracy: 0.9093 - val_loss: 0.3053 - val_accuracy: 0.8887 - lr: 5.0000e-05
Epoch 3/15
1375/1375 [==============================] - 689s 501ms/step - loss: 0.2055 - accuracy: 0.9144 - val_loss: 0.3708 - val_accuracy: 0.8873 - lr: 5.0000e-05
Epoch 4/15
1375/1375 [==============================] - 699s 508ms/step - loss: 0.1764 - accuracy: 0.9243 - val_loss: 0.3758 - val_accuracy: 0.8785 - lr: 2.5000e-05
Epoch 5/15
1375/1375 [==============================] - 710s 516ms/step - loss: 0.1660 - accuracy: 0.9278 - val_loss: 0.2946 - val_accuracy: 0.8880 - lr: 2.5000e-05
Epoch 6/15
1375/1375 [==============================] - 705s 513ms/